In [1]:
from drlqap.taskgenerators import generators
import scipy.optimize
from drlqap.simplesolver import solve_qap_backtracking, solve_qap_maxgreedy, solve_qap_faq, solve_random, solve_partial, solve_partial_random, solve_best_of_k
from drlqap.gurobi import solve_qap_gurobi
import numpy as np
from drlqap.qap import QAP

In [2]:
# change to project root
%cd ..

/home/jupyter-tim/ba-tim


In [3]:
generators

{'minilinear': <drlqap.taskgenerators.LinearTaskGenerator at 0x7fe513770640>,
 'small_random_graphs': <drlqap.taskgenerators.RandomWeightsTaskGenerator at 0x7fe513770700>,
 'medium_random_graphs': <drlqap.taskgenerators.RandomWeightsTaskGenerator at 0x7fe5137707c0>,
 'qaplib_bur26a': <drlqap.taskgenerators.LazyGlobTaskGenerator at 0x7fe513770880>,
 'qaplib_bur26a_normalized': <drlqap.taskgenerators.LazyGlobTaskGenerator at 0x7fe5137708e0>,
 'small_fixed': <drlqap.taskgenerators.LazyGlobTaskGenerator at 0x7fe513770970>,
 'triangle': <drlqap.taskgenerators.SingleTask at 0x7fe513770a00>,
 'increasing_1000': <drlqap.taskgenerators.SteppedTaskGenerator at 0x7fe513770af0>,
 'increasing_simple_1000': <drlqap.taskgenerators.SteppedTaskGenerator at 0x7fe513770b50>,
 'qaplib_all_bur': <drlqap.taskgenerators.LazyGlobTaskGenerator at 0x7fe513770c10>,
 'qaplib_sko_42_64_normalized': <drlqap.taskgenerators.LazyGlobTaskGenerator at 0x7fe513770d60>,
 'qaplib_all_bur_normalized': <drlqap.taskgenerators

In [4]:
qap = generators['medium_random_graphs'].sample()

In [5]:
solve_qap_maxgreedy(qap)

(53.43719807267189, [3, 6, 11, 9, 15, 0, 2, 1, 8, 10, 12, 13, 5, 14, 4, 7])

In [6]:
solve_qap_faq(qap)

(45.87282056393812,
 array([ 8, 13,  9,  0, 15,  5,  6, 14, 11, 10, 12,  1,  3,  2,  4,  7]))

In [55]:
solve_qap_gurobi(qap)

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (linux64)
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads


GurobiError: Model too large for size-limited license; visit https://www.gurobi.com/free-trial for a full license

In [8]:
%timeit solve_qap_faq(qap)

2.15 ms ± 37.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [9]:
!ls runs

a2c_mediumrandoms			 dqn_dense_ms_ec_eps0_mediumrandoms
a2c_ms100x_increasing			 dqn_dense_ms_ec_eps0_mini
a2c_ms100x_mediumrandoms		 dqn_dense_ms_ec_eps0_rni_study
a2c_ms100x_mediumrandoms_cyclic		 dqn_dense_ms_ec_eps0_smallrandoms
a2c_ms100x_mediumrandoms_cyclic_wronglr  dqn_dense_ms_ec_mediumrandoms
a2c_ms100x_mediumrandoms_stepped	 dqn_dense_ms_ec_tai35a
dqn_bi_increasing			 dqn_linkedqap_smallrandoms
dqn_dense_ec_eps0_norm_study		 reinforce_ms100x_mediumrandoms
dqn_dense_ec_eps0_norm_study_2		 reinforce_ms100x_smallrandoms


In [10]:
!ls runs/dqn_dense_ms_ec_eps0_smallrandoms

lr1e-4_s1  lr1e-4_s4  lr1e-5_s3  lr3e-4_s2  lr5e-4_s1  lr5e-4_s4
lr1e-4_s2  lr1e-5_s1  lr1e-5_s4  lr3e-4_s3  lr5e-4_s2
lr1e-4_s3  lr1e-5_s2  lr3e-4_s1  lr3e-4_s4  lr5e-4_s3


In [11]:
from drlqap.evaltools import load_checkpoints
from pathlib import Path

In [12]:
agents = load_checkpoints(Path('runs/a2c_ms100x_mediumrandoms/lr4e-5_s3'))

runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_0.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_1000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_2000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_3000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_4000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_5000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_6000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_7000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_8000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_9000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_10000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_11000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_12000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_13000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_14000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_15000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_

In [13]:
agents[-1].solve(qap)

(48.935399532318115, [7, 6, 1, 15, 12, 4, 14, 11, 10, 13, 3, 0, 8, 2, 5, 9])

## Average results

In [14]:
def evaluate(solver, problem, samples):
    values = []
    for i in range(samples):
        qap = generators[problem].sample()
        v_solver, assignment = solver(qap)
        v = qap.compute_value(assignment)
        if not np.isclose(v, v_solver):
            print (f"solver outputs incorrect value (got {v_solver}, actual {v})")
        values.append(v)
    return np.mean(values)

def evaluate_set(solver, problem):
    values = []
    test_set = generators[problem].test_set()
    for qap in test_set:
        v_solver, assignment = solver(qap)
        v = qap.compute_value(assignment)
        if not np.isclose(v, v_solver):
            print (f"solver outputs incorrect value (got {v_solver}, actual {v})")
        values.append(v)
    return np.mean(values)

In [15]:
evaluate(solve_qap_faq, 'small_random_graphs', 1000)

11.531363

In [16]:
evaluate(agents[-1].solve, 'small_random_graphs', 500)

11.794469

In [17]:
evaluate(solve_qap_faq, 'medium_random_graphs', 1000)

49.450596

In [18]:
evaluate(agents[-1].solve, 'medium_random_graphs', 500)

52.99385

In [19]:
evaluate_set(solve_qap_faq, 'qaplib_all_64_normalized')

32217150.0

In [49]:
evaluate_set(agents[-1].solve, 'qaplib_all_64_normalized')

56063988.0

In [21]:
evaluate_set(solve_qap_faq, 'qaplib_tai35a_normalized')

2514002.0

In [22]:
evaluate_set(agents[-1].solve, 'qaplib_tai35a_normalized')

2755434.0

In [23]:
evaluate_set(agents[-1].solve, 'qaplib_tai35a_normalized')

2755434.0

## Qaplib

In [24]:
qaplib = generators["qaplib_all_64_normalized"].test_set()
qaplib_by_name = {q.name: q for q in qaplib}

In [25]:
agents[-1].solve(qaplib_by_name["bur26c"])

(6153352.0,
 [0,
  2,
  19,
  13,
  18,
  23,
  21,
  25,
  10,
  24,
  14,
  15,
  4,
  8,
  7,
  22,
  6,
  3,
  5,
  20,
  1,
  12,
  17,
  11,
  16,
  9])

# Asymmetry

In [36]:
def mirror(qap):
    a = qap.A
    b = qap.B
    return QAP(b, a, qap.linear_costs.transpose(0,1), 0)
    
def mirror_assignment(assignment):
    out = [None] * len(assignment)
    for i, j in enumerate(assignment):
        out[j] = i
    return out

def mirror_solver(solver):
    def solve_mirrored(qap):
        v, assignment = solver(mirror(qap))
        return v, mirror_assignment(assignment)
    return solve_mirrored

In [37]:
qap_mirrored = mirror(qap)

In [38]:
qap.compute_value([12, 13, 2, 10, 1, 14, 4, 7, 6, 8, 3, 11, 9, 0, 5, 15])

tensor(53.0156)

In [39]:
qap_mirrored.compute_value(mirror_assignment([12, 13, 2, 10, 1, 14, 4, 7, 6, 8, 3, 11, 9, 0, 5, 15]))

tensor(53.0156)

In [40]:
agents[-1].solve(qap)

(48.935399532318115, [7, 6, 1, 15, 12, 4, 14, 11, 10, 13, 3, 0, 8, 2, 5, 9])

In [41]:
v, ass = agents[-1].solve(qap_mirrored)
print(v, ass, qap_mirrored.compute_value(ass))

50.66145747900009 [6, 5, 13, 4, 10, 12, 7, 15, 14, 0, 3, 8, 9, 1, 2, 11] tensor(50.6615)


In [42]:
#evaluate_set(mirror_solver(solve_qap_faq), 'qaplib_all_64_normalized')

In [48]:
evaluate_set(mirror_solver(agents[-1].solve), 'qaplib_all_64_normalized')

38336068.0

In [44]:
evaluate(mirror_solver(agents[-1].solve), 'medium_random_graphs', 50)

52.9594

In [ ]:
evaluate(lambda qap: agents[-1].solve(mirror(qap)), 'small_random_graphs', 500)

In [ ]:
agents[-1].solve(mirror(qaplib_by_name["bur26c"]))

In [ ]:
evaluate_set(agents[-1].solve, 'qaplib_all_bur_normalized')

In [ ]:
evaluate_set(lambda qap: agents[-1].solve(mirror(qap)), 'qaplib_all_bur_normalized')

# 

In [ ]:
qap2 = generators['small_random_graphs'].sample()

In [ ]:
solve_qap_backtracking(qap2)

In [ ]:
%timeit solve_partial_random(qap2, 5)

In [53]:
%timeit agents[-1].solve(qap)

79.6 ms ± 4.35 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
